# ACCESO API 

In [1]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd

## 1. CREAMOS API DE ACCESO 

In [ ]:
api_key="AIisdfjAJFJFAidfjfgnjaiDMMDKKiD" 

In [ ]:
api_service_name = "youtube"
api_version = "v3"

# Creamos la conexión con youtube API
youtube = build(
    api_service_name, api_version, developerKey=api_key)

### 1.1 INDICAMOS PARAMETROS DE BÚSQUEDA 

In [ ]:
TOPICOS = [
    "amnistia", 
    "feminismo", 
    "genero", 
    "monarquia", 
    "salud", 
    "territorio", 
    "violencia_genero", 
    "vivienda", 
    "educacion", 
    "economia", 
    "inmigracion", 
]

In [ ]:
topic = TOPICOS[3]
print("Topico elegido", topic)


In [ ]:
search_response = youtube.search().list(
    q= f"Opiniones españolas sobre {topic} reciente", # q es la query 
    part="id,snippet",
    maxResults= 50, 
    regionCode='ES', 
    # pageToken = search_response['nextPageToken']  #pasa de pagina porque apunta a la siguiente respuesta posible
    #ejecutar page token 3 veces
    
  ).execute()

### 1.1.2 EXTRAEMOS INFORMACION DE LOS VIDEOS 

In [ ]:
def get_video_ids():
    response = []
    for video_response in search_response["items"]:
        video_id = video_response["id"]["videoId"]
        response.append(video_id)
    return response

In [ ]:
from pprint import pprint
def get_comments(video_id):
    comments = []
    try:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText'
        ).execute()
    except Exception:
        print(f"Fallo conseguir comentarios de {video_id}")
        return []

    for item in response['items']:
        try:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            author_channel_id = item['snippet']['topLevelComment']['snippet']['authorChannelId']["value"]
            #author_channel_id = comment['authorChannelId']['value']
            comments.append({"video_id": video_id, "author": author_channel_id, "comment": comment})
        except KeyError as e:
            print(f"Falta algun valor de un comentario, {e}")
            comments.append(None)
            continue
    return comments

In [ ]:
from tqdm.notebook import tqdm 
comment_video = [] 
for video in tqdm(video_id): 
    comments = get_comments(video) 
    comment_video.extend(comments)

In [ ]:
comentarios_inmigracion= pd.to_csv(f'video_comentarios_{topic}.csv')

# 2. PROCESAMIENTO DE DATOS 

## 2.1 PREPOCESAMIENTO DE DATOS 

In [ ]:
valores_nulos = comentarios_inmigracion.isnull().sum()  # Esto devuelve la cantidad de valores nulos por columna
print(valores_nulos)

In [ ]:
# Check data types
comentarios_inmigracion.dtypes

## 2.2 TOKENIZACION 

In [ ]:
#sent_tokenize todo el comentario
comment_st = nltk.tokenize.sent_tokenize(comment)
len(comment_st)

In [ ]:
#word_tokenize tokeniza palabra por palabra
comment_wt = nltk.tokenize.word_tokenize(comment)
len(comment_wt)

### 2.2.1 NORMALIZACION 

In [ ]:
stopwords = nltk.corpus.stopwords.words('spanish')
stopwords = [palabra for palabra in stopwords if 'not' not in palabra]
stopwords

In [ ]:
comment_wt_sin_sw = [word for word in comment_wt if word not in stopwords]
len(comment_wt_sin_sw)

### 2.2.2 FRECUENCIA DE PALABRAS 

In [ ]:
freq = nltk.FreqDist(comment_wt_sin_sw)
freq

In [ ]:
freq_comment= pd.DataFrame(list(freq.items()), columns = ["Word","Frequency"])
freq_comment

In [ ]:
#VISUALIZACION DE FRECUENCIA
plt.figure(figsize = (15,8))
plot = sns.barplot(x  = freq_comment.iloc[:30].Word, y = freq_comment.iloc[:30].Frequency)
for item in plot.get_xticklabels():
    item.set_rotation(90)
plt.show()

## 2.3 STEMIZACION 

In [ ]:
# Importamos esta libreria que nos permite reemplzar caracteres
import re

# Importamos la función que nos permite Stemmizar de nltk y definimos el stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Traemos nuevamente las stopwords
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
#spanish_stopwords.remove('no')
#stopword.remove('not')

In [ ]:
# Recorremos todos los comentarios y le vamos aplicando la Normalizacion y luego el Stemming a cada uno
comment_list=[] 

for comment in comentarios_inmigracion.comment:
    # Vamos a reemplzar los caracteres que no sean letras por espacios, limpiando los numeros 
    comment=re.sub("[^a-zA-Z\s]"," ",str(comment)) #original[^a-zA-Z]
    # Pasamos todo a minúsculas
    comment=comment.lower()
    # Remover caracteres especiales
    comment = re.sub(r'\W', ' ', comment)
    # Remover múltiples espacios con uno solo
    comment = re.sub(r'\s+', ' ', comment, flags=re.I)
    # remover puntuaciones usando expresiones regulares
    comment = re.sub(r'[^\w\s]', '', comment) 
    # #remover comas y comillas 
    # comment = re.sub(comment.replace(',', '').replace("'", ''))
    
    
    # Tokenizar el texto
    # Tokenizamos para separar las palabras del comentario 
    comment=nltk.word_tokenize(comment)
    # Eliminamos las palabras de menos de 3 letras
    comment= [palabra for palabra in comment if len(palabra)>3]
    # Sacamos las Stopwords
    comment = [palabra for palabra in comment if not palabra in stopwords]

    ## Hasta acá Normalizamos, ahora a stemmizar

    # Aplicamos la funcion para buscar la raiz de las palabras
    comment=[stemmer.stem(palabra) for palabra in comment]
    # Por ultimo volvemos a unir el comentario
    comment=" ".join(comment)

    # Vamos armando una lista con todos los comentarios
    comment_list.append(comment)
    # comentarios_inmigracion["comment_normalizado"] = comment_2


In [ ]:
comentarios_inmigracion["comment_stem"] = comment_list
comentarios_inmigracion.head()

### 2.3.1 MODIFICAMOS PARAMETROS DE NORMALIZACION

In [ ]:
# Recorremos todos los comentarios y le vamos aplicando la Normalizacion y luego el Stemming a cada uno
comment_list=[] 
comment_2 = []
for comment in comentarios_inmigracion.comment:
    # Vamos a reemplzar los caracteres que no sean letras por espacios, limpiando los numeros 
    comment=re.sub("[^a-zA-Z]"," ",str(comment))
    # Pasamos todo a minúsculas
    comment=comment.lower()
    # Remover caracteres especiales
    # comment = re.sub(r'\W', ' ', comment)
    # Remover múltiples espacios con uno solo
    comment = re.sub(r'\s+', ' ', comment, flags=re.I)
    # Tokenizar el texto
    # Tokenizamos para separar las palabras del comentario 
    comment=nltk.word_tokenize(comment)
    # Eliminamos las palabras de menos de 3 letras
    comment= [palabra for palabra in comment if len(palabra)>3]
    # Sacamos las Stopwords
    comment = [palabra for palabra in comment if not palabra in stopwords]

    ## Hasta acá Normalizamos, ahora a stemmizar

    # Aplicamos la funcion para buscar la raiz de las palabras
    comment=[stemmer.stem(palabra) for palabra in comment]
    # Por ultimo volvemos a unir el comentario
    comment=" ".join(comment)

    # Vamos armando una lista con todos los comentarios
    comment_2.append(comment)
    # comentarios_inmigracion["comment_normalizado"] = comment_2


### 2.3.2 VECTORIZACION

In [ ]:
list_comment = list(comentarios_inmigracion['comment_stem'].values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Usaremos solo las 1000 palabras con mas frecuencia en todo el corpus para generar los vectores
max_features=1000
# spanish=stopwords.words("spanish")
# Es decir que cada instancia tendrá 1000 features
cou_vec=CountVectorizer(max_features=max_features, stop_words = spanish_stopwords) 



In [ ]:
#Generarnos los vectores para cada COMENTARIO a partir del corpus total. 
matriz_titulos = cou_vec.fit_transform(list_comment)

# Tomamos las palabras
all_words = cou_vec.get_feature_names_out()

# Vizualizamos las 50 palabras mas usadas
print("50 palabras mas usadas: ",all_words[0:50])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transformar el texto en vectores TF-IDF
vectores_tfidf = vectorizer.fit_transform(list_comment)

# Obtener el tamaño del vocabulario
tamaño_vocabulario = len(vectorizer.get_feature_names_out())

# Ver los vectores TF-IDF resultantes 
print(vectores_tfidf) 

#output es:  fila 0, la palabra correspondiente al índice 12289 tiene un valor de TF-IDF 0.4093095079403981


### 2.3.3 OTROS PARAMETROS DE VECTORIZACION 

In [ ]:
vectorizer_2 = TfidfVectorizer(ngram_range=(1, 2), max_df=0.5, min_df=2)


# Transformar el texto en vectores TF-IDF
vectores_tfidf = vectorizer_2.fit_transform(list_comment)

# Obtener el tamaño del vocabulario
tamaño_vocabulario = len(vectorizer_2.get_feature_names_out())

# Ver los vectores TF-IDF resultantes 
print(vectores_tfidf) 
#output  fila 0, la palabra correspondiente al índice 18787 tiene un valor de TF-IDF 0.3394112360296132


In [ ]:
# Convertir la matriz dispersa a un array NumPy
vectores_tfidf_array = vectores_tfidf.toarray()

# Asegurar que el número de filas en vectores_tfidf coincida con la cantidad de filas en comentarios_inmigracion
# (vectores_tfidf_array debe tener la misma cantidad de filas que comentarios_inmigracion)
if vectores_tfidf_array.shape[0] == len(comentarios_inmigracion):
    # Asignar los vectores TF-IDF al DataFrame como una nueva columna
    comentarios_inmigracion['vectores_tfidf'] = list(vectores_tfidf_array)
else:
    print("Las longitudes no coinciden entre la matriz dispersa y el DataFrame.")

## 2.4 LEMMATIZACION 

In [ ]:
# Importamos el lemmatizar de NLTK, y creamos el objeto
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer() 

import nltk
nltk.download('averaged_perceptron_tagger') 

nltk.download("wordnet")

from nltk.corpus import wordnet

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
dataset = pd.read_csv('C:/Users/Usuario/Desktop/ID BOOTCAMPS DS/PROGRAMACION/comentarios_inmigracion.csv')
# dataset = pd.concat([dataset.headline,dataset.is_sarcastic],axis=1)
dataset.dropna(axis=0,inplace=True)

# Traemos nuevamente las stopwords
stopwords = nltk.corpus.stopwords.words('spanish')
#stopword.remove('no')
#stopword.remove('not')

In [ ]:
comentarios_list=[]
for comentario in comentarios_inmigracion.comment:
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    comentario=re.sub("[^a-zA-Z]"," ",str(comentario))
    # Pasamos todo a minúsculas
    comentario=comentario.lower()
    # Tokenizamos para separar las palabras
    comentario=nltk.word_tokenize(comentario)

    # Aplicamos el Lemmatizer (Esto puede tardar un ratito)
    frase_lemma = [wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in comentario]


    # Eliminamos las palabras d emenos de 3 letras
    comentario = [palabra for palabra in comentario if len(palabra)>3]
    # Sacamos las Stopwords
    comentario = [palabra for palabra in comentario if not palabra in stopwords]

    # Por ultimo volvemos a unir el comentario
    comentario=" ".join(comentario)
    #comentarios_inmigracion["titular_normalizado"] = comentarios_list
    comentarios_list.append(comentario)

In [ ]:
comentarios_inmigracion["comment_lemm"] = comentarios_list

# 3. GUARDAMOS CSV

In [ ]:
comentarios_inmigracion.to_csv(f'comentarios_{topic}_procesados.csv')